# Working with Mixtures

In version 2.1, OpenPNM introduced a new *Mixture* class, which as the name suggests, combines the properties of several phases into a single mixture.  The most common example would be diffusion of oxygen in air, which is of course a mixture of $O_2$ and $N_2$ (ignoring humidity and other minor gases like $CO_2$).  The basic premise is that you create normal OpenPNM *Phase* object for each of the pure components, then create a *Mixture* object where you specify the composition of each species.  The notebook gives an overview of how this *Mixture* class works.

In [2]:
import openpnm as op


Start by defining a simple 2D network (for easier visualization):

In [31]:
pn = op.network.Cubic(shape=[4, 4], spacing=0.001)
geo = op.geometry.StickAndBall(network=pn, pores=pn.Ps, throats=pn.Ts)


In principle, you can define the two pure species as ``GenericPhase`` objects, but this leads to problems later since you have to add all the needed physical properties (i.e. molecular weight).  A better option is to use the the pre-defined classes in the OpenPNM.mixture submodule.  Note that this is *not* imported with OpenPNM by default so you must import it explicitly:

In [32]:
from openpnm.phases import mixtures
O2 = mixtures.species.gases.O2(network=pn, name='oxygen')
N2 = mixtures.species.gases.N2(network=pn, name='nitrogen')

With the two 'pure' phases defined, we can now create the mixture phase.  

In [42]:
air = mixtures.GenericMixture(network=pn, components=[N2, O2])
air.add_model(propname='pore.molar_density',
              model=op.models.phases.molar_density.ideal_gas)

SOURCE     : openpnm.phases.mixtures.GenericMixture.__init__ 
TIME STAMP : 2020-03-18 00:14:36,702


Of course, the ``air`` object needs to know the concentration of each species.  The *Mixture* class has a method for setting this.  Note that you only need to specify *N-1* mole fractions and the *N-th* one can be determined.

In [44]:
air.set_mole_fraction(component=O2, values=0.21)

Now we can print the ``air`` object and see how the properties of the mixture are represented:

In [45]:
print(air)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
openpnm.phases.mixtures.GenericMixture : mix_05
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Properties                                    Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.molar_density                               16 / 16   
2     pore.mole_fraction.all                           16 / 16   
3     pore.mole_fraction.nitrogen                      16 / 16   
4     pore.mole_fraction.oxygen                        16 / 16   
5     pore.pressure                                    16 / 16   
6     pore.temperature                                 16 / 16   
7     throat.mole_fraction.all                         24 / 24   
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Labels                                        Assigned Locations
――――――――――――――――――――――――――――――――――――――

As can be seen above, the ``'pore.mole_fraction'`` property has the pure component name appended to the end so we can tell them apart.  We can also look at the values within each array to confirm they are correct:

In [46]:
print(air['pore.mole_fraction.oxygen'])

[0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21 0.21
 0.21 0.21]


In [51]:
print(air['pore.mole_fraction.nitrogen'])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [52]:
air.update_mole_fractions()

In [53]:
air['pore.molar_density']

array([40.8945824, 40.8945824, 40.8945824, 40.8945824, 40.8945824,
       40.8945824, 40.8945824, 40.8945824, 40.8945824, 40.8945824,
       40.8945824, 40.8945824, 40.8945824, 40.8945824, 40.8945824,
       40.8945824])

In [21]:
print(air['pore.mole_fraction.nitrogen'])

[0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79 0.79
 0.79 0.79]


In [20]:
phys = op.physics.GenericPhysics(network=pn, phase=air, geometry=geo)